In [9]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [17]:
%env NUMBA_WARNINGS 1
%env NUMBA_NUM_THREADS 24

import ctypes
mkl_rt = ctypes.CDLL('libmkl_rt.so')
print(mkl_rt.mkl_get_max_threads())
mkl_get_max_threads = mkl_rt.mkl_get_max_threads
def mkl_set_num_threads(cores):
    mkl_rt.mkl_set_num_threads(ctypes.byref(ctypes.c_int(cores)))
mkl_set_num_threads(24)
print(mkl_get_max_threads())

env: NUMBA_WARNINGS=1
env: NUMBA_NUM_THREADS=24
16
24


In [18]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import os
import pickle as pkl
from telepyth import TelepythClient
import time

from approximate import APPROX
from experiment import relative_errors
from dataset import make_dataset
from visualize import plot

# USPS

In [19]:
approx_types = ['G', 'Gort', 'ROM', 'QMC', 'GQ', 'B']
# halton sequence generator cannot generate large enough sequences for datasets CIFAR100 and LEUKEMIA
# same with GQ method, where the scipy Hermite polynomials roots generator also restricts the max dataset dim.
approx_types_large_d = ['G', 'Gort', 'ROM', 'B']

In [20]:
datasets = ['USPS', 'LETTER', 'Powerplant', 'MNIST', 'CIFAR100', 'LEUKEMIA']

In [21]:
# datasets = ['LEUKEMIA'] #['LETTER', 'Powerplant', 'MNIST', 'CIFAR100', 'LEUKEMIA']

In [22]:
kernels = ['Arccos 1', 'RBF', 'Arccos 0']

In [ ]:
%%time
%%telepyth
sample_params = None  # [1, 2, 1, 0, 1, 5, 50]
for name in datasets:
    X, Y, params = make_dataset(name, sample_params)
    errs = {}
    start_deg, max_deg, runs, shift, step, _, _ = params
    if X.shape[1] > 784:
        approx_types = approx_types_large_d
        
    for kernel in kernels:
        errs[kernel] = relative_errors(X, Y, kernel, approx_types, start_deg, max_deg, step, shift, runs)
        
    directory = 'results/%s/' % name
    if not os.path.exists(directory):
        os.makedirs(directory)

    with open(directory + '%s_%r_%r' % (name, kernels, params), 'wb') as f:
        pkl.dump(errs, f)
        
    plot(errs)

  0%|          | 0/6 [00:00<?, ?it/s]

G


 17%|█▋        | 1/6 [03:19<16:37, 199.46s/it]

Gort


 33%|███▎      | 2/6 [07:25<14:13, 213.36s/it]

...

Gort


 33%|███▎      | 2/6 [18:39<36:41, 550.33s/it]

ROM


 50%|█████     | 3/6 [27:08<26:54, 538.05s/it]

QMC
